# Main-code for House-Amneties-Neighbourhood-Crime-Income

In [ ]:
# 3rd party imports
import os
import json
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pylab as plt
from bs4 import BeautifulSoup
#!pip install pandas openpyxl


# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False


In [ ]:
#Load CSV 
tn_df = pd.read_csv("Toronto-Neigh.csv", encoding='Windows-1252')
tnT_df=tn_df.T

# View DataFrame
tn_df.head()

## Filtering and Cleaning the dataframe to obtain the required data set.

In [ ]:
# Extract the first data row (index 1, which is the second row of the sheet)
first_row = tn_df.T.iloc[0].astype(str)

# Keywords to search for
keywords = [
    'Income statistics in 2020 for the population aged 15 years and over in private households - 25% sample data',
   # 'Income', 
    'tax',
    'after-tax income',
    'after-tax income in 2020',
    'Median total income'
]

# Initialize a dictionary to store all indexes for each keyword
all_keyword_indexes = {}

# Find keywords in the first row (case-insensitive matching) and their corresponding indexes
for keyword in keywords:
    # Find all indexes for the keyword
    indexes = first_row[first_row.str.contains(keyword, case=False, na=False)].index.tolist()
    
    # If keyword is found, store the indexes
    if indexes:
        # Store the indexes for the keyword
        all_keyword_indexes[keyword] = indexes
        
        # Calculate and print only the min and max indexes
        min_index = min(indexes)  # Get the minimum index
        max_index = max(indexes)  # Get the maximum index
        print(f"Keyword '{keyword}' found at min index: {min_index} and max index: {max_index}")



In [ ]:
# Access the actual values at columns 60 and 1949 (first row)
value_at_index_60 = tn_df.T.iloc[0,92]  # 61st column, first row (index 0)
value_at_index_1949 = tn_df.T.iloc[0, 424]  # 1950th column, first row (index 0)

# Optionally, get the column names at those indices
column_name_60 = tn_df.T.columns[92]  # 61st column name
column_name_1949 = tn_df.T.columns[424]  # 1950th column name

# Print the results
print(f"Value at column {column_name_60} (index 60): {value_at_index_60}")
print(f"Value at column {column_name_1949} (index 1949): {value_at_index_1949}")


In [ ]:
# Step 1: Extract the first column (assuming it contains labels or identifiers)
first_column = tn_df.T.iloc[:, 0]

# Step 2: Filter the columns based on index range (between 60 and 168)
filtered_columns = tn_df.T.iloc[:, 60:167]  # Select columns from index 60 to 168 (inclusive of 60, exclusive of 169)

# Step 3: Create a new DataFrame with the first column and the filtered columns
filtered_df = pd.concat([first_column, filtered_columns], axis=1)

# Step 4: Print the resulting filtered DataFrame
(filtered_df)


In [ ]:
# Step 1: Reset the row index and add it as a column (this step you already have)
filtered_df_reset = filtered_df.reset_index()

# Step 2: Set the row index column to the first column (this you already did)
filtered_df_reset.set_index(filtered_df_reset.columns[0], inplace=True)

# Step 3: Reset column headers to default (0, 1, 2, ...)
# This step will reset the column headers (the numeric indices you mentioned)
filtered_df_reset.columns = range(filtered_df_reset.shape[1])

# Confirmation message
print("Row index has been moved to the first column, and column headers have been reset.")
filtered_df_reset

In [ ]:
# Step 1: Reset the index, which makes the current index a column
filtered_df_reset1 = filtered_df_reset.reset_index()

# Optional Step 2: Set a specific column as the new index (if you need a specific index)
# df_reset.set_index('your_column_name', inplace=True)

# Confirmation message
print("Index column has been moved to the first column, and row index has been reset.")

filtered_df_reset1= filtered_df_reset1.rename(columns={"index": "Neighbourhood"})
filtered_df_reset1

## Loading the file from the previous pipeline step.

In [ ]:
#second file
hos_t=pd.read_csv("Toronto_houses_with_crime_data_0.5km.csv")
hos_t

### Combining the data_sets

In [ ]:
df=hos_t
df['Neighbourhood'] = df['Neighbourhood'].str.lower()
column_name = 'Neighbourhood'

# Get unique entries in the column
unique_entries = df[column_name].dropna().unique()

# Convert to a list if needed (or you can leave it as an array)
unique_entries_list = unique_entries.tolist()

# Display the unique entries
len(unique_entries_list)

In [ ]:
df=filtered_df_reset1
df['Neighbourhood'] = df['Neighbourhood'].str.lower()
column_name = 'Neighbourhood'

# Get unique entries in the column
unique_entries = df[column_name].dropna().unique()

# Convert to a list if needed (or you can leave it as an array)
unique_entries_list = unique_entries.tolist()

# Display the unique entries
len(unique_entries_list)


In [ ]:
# Load the two DataFrames
df1 = hos_t
df2 = filtered_df_reset1

# Specify the column with unique entries in df1 and the corresponding column in df2
unique_column_df1 = 'Neighbourhood'
matching_column_df2 = 'Neighbourhood'

# Clean the 'Neighbourhood' column in both DataFrames
#df1[unique_column_df1] = df1[unique_column_df1].str.strip().str.lower().replace({'\'': '', ',': '': ''}, regex=True)
df1[unique_column_df1] = df1[unique_column_df1].str.strip().str.lower().replace({'\'': ' ', ',': ' ', '-': ' ','`': ' '}, regex=True).str.replace(r'\s+', ' ', regex=True)

df2[matching_column_df2] = df2[matching_column_df2].str.strip().str.lower().replace({'\'': ' ', ',': ' ', '-': ' ','`': ' '}, regex=True).str.replace(r'\s+', ' ', regex=True)

df1[unique_column_df1] = df1[unique_column_df1].str.replace(r'\bst\.\b', 'st ', regex=True, case=False)
df2[matching_column_df2] = df2[matching_column_df2].str.replace(r'st\.', 'st ', regex=True, case=False)

df1[unique_column_df1] = df1[unique_column_df1].str.replace(r'\s+', ' ', regex=True)
df2[matching_column_df2] = df2[matching_column_df2].str.replace(r'\s+', ' ', regex=True)


# Step 1: Get unique entries from df1 (now cleaned)
unique_entries = df1[unique_column_df1].dropna().unique()

# Step 2: Filter rows in df2 where the matching column values are in unique_entries
filtered_df2 = df2[df2[matching_column_df2].isin(unique_entries)]

# Display the filtered dataframe
filtered_df2


In [ ]:
# Get entries in df1 that are not in df2
unmatched_entries = set(unique_entries) - set(df2[matching_column_df2].unique())
print("Unmatched entries:", unmatched_entries)


### Getting only the subset of income dataset

In [ ]:
# Select the specific row in df1 that you want to use as column headers
# For example, if it's the first row (index 0):
df1= filtered_df_reset
new_headers = df1.iloc[0]
len(new_headers)

df2= filtered_df2
# # Update df2's column names with these new headers
# Keep the first column header of df2 unchanged
df2_columns = [df2.columns[0]] + list(new_headers)

# Update df2's columns
df2.columns = df2_columns

# # Confirm the change
# print("Updated column names of df2:")
income_1=df2
income_1



In [ ]:
# Select and convert the necessary columns to lists
columns_to_keep = ['Neighbourhood'] + ['    Average total income in 2020 among recipients ($)'] + ['    Average after-tax income in 2020 among recipients ($)'] #+ income_1.iloc[:,31:80]

# Use the columns_to_keep list to select those columns from the DataFrame
income = income_1[columns_to_keep]

# Display the new DataFrame
(income)


In [ ]:
#segregating gross-aftertax income's
# Group 1: Columns 1 to N (excluding the first column)
# Adjust the range of columns as necessary
df_1 = income_1.iloc[:, 31:47]  # First half of remaining columns
df_2 = income_1.iloc[:, 47:63]   # Second half of remaining columns
df_3 = income_1.iloc[:, 63:80]   # Second half of remaining columns

# # Step 3: Add the 'Neigh' column back to each new DataFrame
df_1 = pd.concat([income,df_1], axis=1) #gross
df_2 = pd.concat([income,df_2], axis=1) #after-tax
df_3 = pd.concat([income,df_3],axis=1)  #employement


#### Combining the income braackets to define 6 classes

In [ ]:
##Gross-income of individuals

# Add the values of two columns (e.g., 'column1' and 'column2') and assign the result to a new column
avg_expense = 1530*12

df_1['Class 6G'] = pd.to_numeric(df_1.iloc[:,7], errors='coerce') + pd.to_numeric(df_1.iloc[:,8], errors='coerce')
df_1['Class 5G'] = pd.to_numeric(df_1.iloc[:,9], errors='coerce') + pd.to_numeric(df_1.iloc[:,10], errors='coerce') +pd.to_numeric(df_1.iloc[:,11], errors='coerce')
df_1['Class 4G'] = pd.to_numeric(df_1.iloc[:,12], errors='coerce') + pd.to_numeric(df_1.iloc[:,13], errors='coerce') 
df_1['Class 3G'] = pd.to_numeric(df_1.iloc[:,14], errors='coerce') + pd.to_numeric(df_1.iloc[:,15], errors='coerce') +pd.to_numeric(df_1.iloc[:,16], errors='coerce')
df_1['Class 2G'] = pd.to_numeric(df_1.iloc[:,17], errors='coerce') 
df_1['Class 1G'] = pd.to_numeric(df_1.iloc[:,18], errors='coerce') 


# Print the DataFrame to confirm the new column
(df_1)


In [ ]:
## After tax reduction income

# Add the values of two columns (e.g., 'column1' and 'column2') and assign the result to a new column
avg_expense = 1530

df_2['Class 6G_T'] = pd.to_numeric(df_2.iloc[:,7]) + pd.to_numeric(df_2.iloc[:,8])
df_2['Class 5G_T'] = pd.to_numeric(df_2.iloc[:,9]) + pd.to_numeric(df_2.iloc[:,10]) +pd.to_numeric(df_2.iloc[:,11])
df_2['Class 4G_T'] = pd.to_numeric(df_2.iloc[:,12]) + pd.to_numeric(df_2.iloc[:,13]) 
df_2['Class 3G_T'] = pd.to_numeric(df_2.iloc[:,14]) + pd.to_numeric(df_2.iloc[:,15]) +pd.to_numeric(df_2.iloc[:,16])
df_2['Class 2G_T'] = pd.to_numeric(df_2.iloc[:,17]) 
df_2['Class 1G_T'] = pd.to_numeric(df_2.iloc[:,18]) 

# Print the DataFrame to confirm the new column
(df_2)


In [ ]:
# Step 1: Ensure "Neighbourhood" is the index in both DataFrames
df_1.set_index('Neighbourhood', inplace=True)  # Modify df_1 in place
hos_t.set_index('Neighbourhood', inplace=True)  # Modify hos_t in place
# df_1.set_index('Neighbourhood', inplace= False)
# hos_t.set_index('Neighbourhood', inplace= False)

# Step 2: Select the last 6 columns from df_1
last_6_columns_df1 = df_1.iloc[:, -6:]

# Step 3: Merge these columns into hos_t based on the "Neighbourhood" index
hos_t = hos_t.merge(last_6_columns_df1, how='left', left_index=True, right_index=True)

# Step 4: Reset the index if you want the "Neighbourhood" back as a column
hos_t.reset_index(inplace=True)

# Step 5: Display the updated hos_t DataFrame
(hos_t)


In [ ]:
# Drop rows that have NaN in specific columns 
final_df = hos_t.dropna(subset=['Class 6G'])

# Display the DataFrame after dropping rows
(final_df)


In [ ]:
# Step 1: Ensure "Neighbourhood" is the index in both DataFrames
df_2.set_index('Neighbourhood', inplace=True)  # Modify df_1 in place
hos_t.set_index('Neighbourhood', inplace=True)  # Modify hos_t in place
# df_1.set_index('Neighbourhood', inplace= False)
# hos_t.set_index('Neighbourhood', inplace= False)

# Step 2: Select the last 6 columns from df_1
last_6_columns_df2 = df_2.iloc[:, -6:]

# Step 3: Merge these columns into hos_t based on the "Neighbourhood" index
hos_t = hos_t.merge(last_6_columns_df2, how='left', left_index=True, right_index=True)

# Step 4: Reset the index if you want the "Neighbourhood" back as a column
hos_t.reset_index(inplace=True)

# Step 5: Display the updated hos_t DataFrame
(hos_t)


In [ ]:
# Drop rows that have NaN in specific columns (e.g., 'Column1', 'Column2', ...)
final_df2 = hos_t.dropna(subset=['Class 6G_T'])

# Display the DataFrame after dropping rows
(final_df2)


In [ ]:
final_df2.to_csv('Toronto_houses_with_crime_income_data_0.5km.csv', index=False)